In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn import preprocessing

# Preprocessing 

In [2]:
raw_csv_data= np.loadtxt(r"C:\Users\RAJSHREE\Downloads\Audiobooks_data.csv" , delimiter=',')

In [3]:
unscaled_inputs_all= raw_csv_data[:,1:-1]
targets_all= raw_csv_data[:,-1]

In [4]:
num_one_targets= int(np.sum(targets_all))
zero_targets_counter= 0 
indices_to_remove= []

for i in range (targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors= np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors= np.delete (targets_all, indices_to_remove, axis=0)

In [5]:
scaled_inputs= preprocessing.scale(unscaled_inputs_equal_priors)

In [7]:
shuffled_indices= np.arange (scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs= scaled_inputs[shuffled_indices]
shuffled_targets= targets_equal_priors[shuffled_indices]

In [10]:
samples_count= shuffled_inputs.shape[0]

train_samples_count= int(0.8*samples_count)
validation_samples_count= int(0.1*samples_count)
test_samples_count= samples_count- train_samples_count- validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1768.0 3579 0.49399273540095
224.0 447 0.5011185682326622
245.0 448 0.546875


In [11]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# Loading the data

In [13]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# Model

In [20]:
input_size= 10
output_size= 2
hidden_layer_size= 50

model = tf.keras.Sequential([ 
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size= 100
max_epochs= 100
early_stopping= tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks= [early_stopping],
          validation_data= (validation_inputs, validation_targets),
          verbose= 2
          )

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.5224 - accuracy: 0.7734 - val_loss: 0.4087 - val_accuracy: 0.8613
Epoch 2/100
3579/3579 - 0s - loss: 0.3503 - accuracy: 0.8765 - val_loss: 0.3361 - val_accuracy: 0.8770
Epoch 3/100
3579/3579 - 0s - loss: 0.3110 - accuracy: 0.8885 - val_loss: 0.3076 - val_accuracy: 0.8904
Epoch 4/100
3579/3579 - 0s - loss: 0.2917 - accuracy: 0.8930 - val_loss: 0.2923 - val_accuracy: 0.8971
Epoch 5/100
3579/3579 - 0s - loss: 0.2796 - accuracy: 0.8963 - val_loss: 0.2799 - val_accuracy: 0.8971
Epoch 6/100
3579/3579 - 0s - loss: 0.2697 - accuracy: 0.9003 - val_loss: 0.2746 - val_accuracy: 0.8993
Epoch 7/100
3579/3579 - 0s - loss: 0.2635 - accuracy: 0.9033 - val_loss: 0.2656 - val_accuracy: 0.8993
Epoch 8/100
3579/3579 - 0s - loss: 0.2583 - accuracy: 0.9064 - val_loss: 0.2585 - val_accuracy: 0.9016
Epoch 9/100
3579/3579 - 0s - loss: 0.2540 - accuracy: 0.9067 - val_loss: 0.2526 - val_accuracy: 0.9038
Epoch 10/100
3579/3579 - 0

# Testing the model

In [21]:
test_loss, test_accuracy= model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 324us/sample - loss: 0.2712 - accuracy: 0.8929


In [22]:
print('\n Test loss {0:.2f} Test Accuracy {1:.2f}%'.format(test_loss, test_accuracy*100.))


 Test loss 0.27 Test Accuracy 89.29%
